# Align to combined macaque and canine reference

In [9]:
bp = "PRJNA665227"
fastq_folder = f"/home/laura/projects/virus-watch-data/{bp}/raw"
sample_batch_file = f"{fastq_folder}/batch.txt"

In [10]:
# Ensembl version 110
canine_cdna = "/home/laura/projects/virus-watch-data/canine_ref/Canis_lupus_familiaris.ROS_Cfam_1.0.cdna.all.fa.gz"
macaque_cdna = "/home/laura/projects/virus-watch-data/rhesus_ref/Macaca_mulatta.Mmul_10.cdna.all.fa.gz"

canine_macaque_fasta = "/home/laura/projects/virus-watch-data/canine_macaque_ref/combined_transcriptomes.cdna.all.fa.gz"

kallisto = "/home/laura/projects/kallisto/build/src/kallisto"
bustools = "/home/laura/projects/bustools/build/src/bustools"

### Align to PalmDB

Generate virus index:

In [ ]:
!cat $canine_cdna $macaque_cdna > $canine_macaque_fasta

In [11]:
virus_fasta = "/home/laura/projects/virus-watch-data/virus_ref/uniques_noduplicates.fa"
virus_index = "/home/laura/projects/virus-watch-data/virus_ref/kallisto_aa/noduplicates_rhesus_canine_cdna.idx"
virus_t2g = "/home/laura/projects/virus-watch-data/virus_ref/nodup_clu_t2g.txt"

In [6]:
# Generate virus reference index (with default d-list-overhang = 3)
!/usr/bin/time -v $kallisto index \
    -t 30 \
    --aa \
    --d-list=$canine_macaque_fasta \
    -i $virus_index \
    $virus_fasta


[index] --d-list-overhang was set to 3 (with --aa, the d-list overhang must be >= 3)
[build] loading fasta file /home/laura/projects/virus-watch-data/virus_ref/uniques_noduplicates.fa
[build] k-mer length: 31
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Finished
CompactedDBG::build(): Estimated number of k-mers occurring at least once: 37641510
CompactedDBG::build(): Estimated number of minimizer occurring at least once: 7877811
CompactedDBG::filter(): Processed 87630084 k-mers in 296561 reads
CompactedDBG::filter(): Found 37508820 unique k-mers
CompactedDBG::filter(): Number of blocks in Bloom filter is 257317
CompactedDBG::construct(): Extract approximate unitigs (1/2)
CompactedDBG::construct(): Extract approximate unitigs (2/2)
CompactedDBG::construct(): Closed all input files

CompactedDBG::construct(): Splitting unitigs (1/2)

CompactedDBG::cons

Align to PalmDB and correct barcodes using host onlist:

In [12]:
out_folder = f"{fastq_folder}/../full_index_cdna/virus"

In [13]:
sample_batch_file

'/home/laura/projects/virus-watch-data/PRJNA665227/raw/batch.txt'

In [14]:
%%time
!/usr/bin/time -v $kallisto bus \
        -i $virus_index \
        -o $out_folder \
        --aa \
        -t 40 \
        -B $sample_batch_file \
        --batch-barcodes \
        -x 0,0,12:0,12,20:1,0,0


[bus] will try running read files supplied in batch file
[bus] Note: Strand option was not specified; setting it to --unstranded for specified technology
[index] k-mer length: 31
[index] number of targets: 296,561
[index] number of k-mers: 37,546,591
[index] number of distinguishing flanking k-mers: 4,945
[quant] running in single-end mode
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698516_1.fastq.gz
[quant] will process file 2: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698516_2.fastq.gz
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698524_1.fastq.gz
[quant] will process file 2: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698524_2.fastq.gz
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698536_1.fastq.gz
[quant] will process file 2: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698536_2.fastq.gz
[quant] will process 

In [15]:
host_out_folder = f"/home/laura/projects/virus-watch-data/{bp}/full_index_stringent_dlist_flank1/host_canine"

In [16]:
%%time
!$bustools sort \
    -m 4G \
    -t 40 \
    -o $out_folder/output_sorted.bus \
    $out_folder/output.bus

!$bustools correct \
    -w $host_out_folder/bustools_whitelist.txt \
    -o $out_folder/output_sorted_corrected.bus \
    $out_folder/output_sorted.bus

!$bustools sort \
    -m 4G \
    -t 40 \
    -o $out_folder/output_sorted_corrected_sorted.bus \
    $out_folder/output_sorted_corrected.bus

!$bustools count \
    --genecounts \
    -o $out_folder/bustools_count/ \
    -g $virus_t2g \
    -e $out_folder/matrix.ec \
    -t $out_folder/transcripts.txt \
    $out_folder/output_sorted_corrected_sorted.bus

partition time: 3.99s
partition time: 2.75s
Read in 239560117 BUS records
reading time 9.61s
sorting time 56.84s
writing time 0s
Found 775516 barcodes in the on-list
Processed 68228931 BUS records
In on-list = 14501766
Corrected    = 15965418
Uncorrected  = 37761747
partition time: 0.85s
 all fits in buffer
Read in 30467184 BUS records
reading time 0.54s
sorting time 6.44s
writing time 1.28s
CPU times: user 3.88 s, sys: 876 ms, total: 4.76 s
Wall time: 3min 39s
